In [ ]:
# sequences are from:
# https://pubmed.ncbi.nlm.nih.gov/19858363/

# Documentation progress
- ~~data import~~
- ~~belgian plots~~
- ~~skyline plots~~
- ~~violin plots~~
- ~~mutation calculation~~
- ~~mutation plot~~
- ~~mutation t-tests~~

I believe this is done now, but haven't tested some of it. This code is rough and not in a shape intended for re-use on other projects

In [ ]:
import pandas as pd
import numpy as np
import math
import os
import seaborn as sns

from datetime import datetime

from Bio import SeqIO

from numpy import isnan, nan, sqrt
from numpy import nanpercentile as percentile

from scipy import stats as stats

from types import SimpleNamespace

import matplotlib as mp
mp.use('Agg')
mp.rcParams["font.sans-serif"].insert(0,"Arial")
mp.rcParams["font.family"] = "sans-serif"
mp.rcParams["pdf.fonttype"] = 42 # use TrueType fonts when exporting PDFs
                                 # (embeds most fonts - this is especially
                                 #  useful when opening in Adobe Illustrator)
mp.rcParams['font.size'] = 11
mp.rcParams['xtick.direction'] = 'out'
mp.rcParams['ytick.direction'] = 'out'
mp.rcParams['legend.fontsize'] = 14
mp.rcParams['grid.color'] = ".8"
mp.rcParams['grid.linestyle'] = '-'
mp.rcParams['grid.linewidth'] = 1
# mp.rcParams['figure.constrained_layout.use'] = True
mp.use('Agg')

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# import warnings
# warnings.filterwarnings("ignore")

rx_color = "red"
bg_color = "blue"
dc_color = "darkgoldenrod"

In [ ]:
""" 
Creates Class CellType which represents a real world cell type and encapsulates any datasets that are generated from those cells

the input 'files' here is a tuple with the form (ex, in), replicates = True means files should have 4 items in it, but is not required.
the 'pps' input is the pairing probabilities and functions exactly like tables does
"""

class CellType: 
    def __init__(self, name, files=None, crop=None):
            
        if files is not None:
            if len(files) == 4:
                self.replicates = True
            else:
                self.replicates = False
        
        self.name = name
        
        self._ex = SimpleNamespace()
        self._in = SimpleNamespace()
        
        if crop is not None:
            self._ex.table = pd.read_csv(files[0], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])[crop[0]:crop[1]]
            self._in.table = pd.read_csv(files[1], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])[crop[0]:crop[1]]
        else:
            self._ex.table = pd.read_csv(files[0], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])
            self._in.table = pd.read_csv(files[1], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])
        
        self._ex.name = self.name + "-ex"
        self._in.name = self.name + "-in"
        
        for x in (self._ex, self._in):
            x.seq = x.table.Sequence
            x.num = x.table.Nucleotide
            x.profile = np.array(x.table.Norm_profile)
            x.stderr = np.array(x.table.Norm_stderr)
            x.depth = np.array(x.table.Modified_effective_depth)
            
        if self.replicates:
            self._ex_rep = SimpleNamespace()
            self._in_rep = SimpleNamespace()
            
            if crop is not None:
                self._ex_rep.table = pd.read_csv(files[2], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])[crop[0]:crop[1]]
                self._in_rep.table = pd.read_csv(files[3], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])[crop[0]:crop[1]]
            else:
                self._ex_rep.table = pd.read_csv(files[2], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])
                self._in_rep.table = pd.read_csv(files[3], sep="\t", usecols=["Nucleotide", "Sequence", "Norm_profile", "Norm_stderr", "Modified_effective_depth"])
            
            self._ex_rep.name = self.name + "-ex-replicate"
            self._in_rep.name = self.name + "-in-replicate"
            
            for x in (self._ex_rep, self._in_rep):
                x.seq = x.table.Sequence
                x.num = x.table.Nucleotide
                x.profile = np.array(x.table.Norm_profile)
                x.stderr = np.array(x.table.Norm_stderr)
                x.depth = np.array(x.table.Modified_effective_depth)
                x.med_adjusted = np.nanmedian(x.profile) - x.profile 
            
            self.match_nans(self._in.profile, self._in_rep.profile)
            self.match_nans(self._ex.profile, self._ex_rep.profile)
            
            # copy the original namespaces for where we're going to store the merged equivalent
            self._ex_merged = SimpleNamespace(**self._ex.__dict__)
            self._in_merged = SimpleNamespace(**self._in.__dict__)
            
            # average the replicate and the original for each profile into the merged namespace
            self._ex_merged.profile = (self._ex.profile + self._ex_rep.profile)/2
            self._in_merged.profile = (self._in.profile + self._in_rep.profile)/2
            
            # calculate the uncertainties
            self._ex_merged.stderr = ((self._ex.stderr**2 + self._ex_rep.stderr**2)**.5)/2
            self._in_merged.stderr = ((self._in.stderr**2 + self._in_rep.stderr**2)**.5)/2
            
            self._ex_merged.name = self._ex.name + "-combined"
            self._in_merged.name = self._in.name + "-combined"
            
            # TODO: check if this depth calculation is kosher
            self._ex_merged.depth = self._ex.depth + self._ex_rep.depth
            self._in_merged.depth = self._in.depth + self._in_rep.depth
            
            # TODO: remove this, I don't think it's used anywhere? Check that and if not, it can go
            self._ex_merged.med_adjusted = np.nanmedian(self._ex_merged.profile) - self._ex_merged.profile
            self._in_merged.med_adjusted = np.nanmedian(self._in_merged.profile) - self._in_merged.profile
    
    ''' normalize the data using medians to the median value of the specified set (hek._in_merged, probably) '''
    def median_normalize(self, norm_to):
        
        if self.replicates:
            datasets = (self._in,
                        self._in_rep,
                        self._in_merged,
                        self._ex,
                        self._ex_rep,
                        self._ex_merged,)
        else:
            datasets = (self._in,
                        self._ex)
            
        for x in datasets:
            x.med_normed = x.profile*(np.nanmedian(norm_to.profile)/np.nanmedian(x.profile))
            # NOTE: is this the right thing to be doing with the errors? We're just scaling by the ratio so it should be fine, right?
            x.med_normed_stderr = x.stderr*(np.nanmedian(norm_to.profile)/np.nanmedian(x.profile))
        
    
    ''' output a .map file for the specified dataset into the specified folder, if mask is provided, mask values into np.nans to match each other '''
    def to_map_file(self, dataset_string, fileout, data='profile', mask=None):
        
        if self.replicates:
            datasets = {f'{self.name}-in': self._in,
                        f'{self.name}-in-replicate': self._in_rep,
                        f'{self.name}-in-combined': self._in_merged,
                        f'{self.name}-ex': self._ex,
                        f'{self.name}-ex-replicate': self._ex_rep,
                        f'{self.name}-ex-combined': self._ex_merged}
        else:
            datasets = {f'{self.name}-in': self._in,
                        f'{self.name}-ex': self._ex}
            
        dataset = datasets[dataset_string]
        
        if data == 'profile':
            data = dataset.profile
            errs = dataset.stderrs
        elif data == 'med_normed':
            data = dataset.med_normed
            errs = dataset.med_normed_stderr
        else:
            raise Exception('unknown data table for map file conversion')
        
        # file-ify it into a .map file
        with open(fileout, 'w') as f:
            # each line of the map file has the following structure:
            # num-->shape-->stderr-->nuc
            for i, shape in enumerate(data):
                
                err = errs[i]
                
                # adjust our nan values to be what .map files expect
                if np.isnan(shape):
                    shape = '-999'
                    err = '0'
                
                f.write(f"{dataset.num[i]}\t{shape}\t{str(err)}\t{dataset.seq[i]}\n")
                
    ''' match the nans or -999 values across two datasets, input should be some shape data'''            
    @staticmethod
    def match_nans(data1, data2):            
        nan_mask = np.isnan(data1) | np.isnan(data2)
        # just in case there's -999s as well, for whatever reason (we don't want these because they mess with calculations)
        nan_mask = (data1 == -999) | (data2 == -999) | nan_mask

        data1[nan_mask] = np.nan
        data2[nan_mask] = np.nan
        
    
    ''' add extra nans around the nan regions if a certain percentage of that window is nan'''
    ''' outputs an array of tuples in the form (shape_data, errs), in the order rep1, rep2, merged for both in and ex data'''
    def threshold_nans(self, window_size=51, nan_threshold=.25, dataset="profile", inplace=False):
        if dataset == "med_normed":
            datasets = (
                self._in.med_normed,
                self._in_rep.med_normed,
                self._in_merged.med_normed,
                self._ex.med_normed,
                self._ex_rep.med_normed,
                self._ex_merged.med_normed
            )
            errs = (
                self._in.med_normed_stderr,
                self._in_rep.med_normed_stderr,
                self._in_merged.med_normed_stderr,
                self._ex.med_normed_stderr,
                self._ex_rep.med_normed_stderr,
                self._ex_merged.med_normed_stderr
            )   
        elif dataset == "profile":
            datasets = (
                self._in.profile,
                self._in_rep.profile,
                self._in_merged.profile,
                self._ex.profile,
                self._ex_rep.profile,
                self._ex_merged.profile)
            errs = (
                self._in.stderr,
                self._in_rep.stderr,
                self._in_merged.stderr,
                self._ex.stderr,
                self._ex_rep.stderr,
                self._ex_merged.stderr)
        else:
            raise Exception(f"invalid dataset string: {dataset}")
            
        # initiate the output array
        output = [None]*len(datasets)
            
        # for each dataset and errorset
        for i, (data, err) in enumerate(zip(datasets, errs)):
            # there is probably a more elegant way to do this, but this is fine
            if not inplace:
                data = data.copy()
                err = err.copy()
            
            # create a list of the ratios of nans in that window
            nan_ratios = np.array([np.count_nonzero(np.isnan(data[j-(window_size//2):j+(window_size)//2]))/window_size for j in range(len(data))])
            nan_mask = [nan_ratios > .25]
            
            # every true position in nan_mask should be a nan in both datasets
            data[nan_mask] = np.nan
            err[nan_mask] = np.nan

            output[i] = (data, err)
        
        return output
                
                
            
dir_profiles = "~/compare_reactivities/profiles"                    

vero_files = (
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-ex_CHLSAB2_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-in_CHLSAB2_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-replicate-ex_CHLSAB2_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-replicate-in_CHLSAB2_MALAT1_profile.txt")
)
hek_files = (
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-in_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-replicate-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-replicate-in_HUMAN_MALAT1_profile.txt")
)
a549_files = (
    os.path.join(dir_profiles, "MALAT1-a549-amp-ex-all-1_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-a549-amp-in-all-1_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-a549-amp-all-replicate-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-a549-amp-all-replicate-in_HUMAN_MALAT1_profile.txt")
)

# altered shapemapper runs
# NOTE: temporarily adding ex and ex_rep to these even though that doesn't exist, just filling it in with the original ex as a stand-in
hek_a_files = (
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-in-altered_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-replicate-in-altered_HUMAN_MALAT1_profile.txt")
)
vero_a_files = (
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-in-altered_CHLSAB2_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-vero-amp-all-replicate-in-altered_CHLSAB2_MALAT1_profile.txt")
)
a549_a_files = (
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-a549-amp-all-in-altered_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-hek-amp-all-ex_HUMAN_MALAT1_profile.txt"),
    os.path.join(dir_profiles, "MALAT1-a549-amp-all-replicate-in-altered_HUMAN_MALAT1_profile.txt")
)

# create each dataset to be used
hek = CellType("hek", hek_files)
vero = CellType("vero", vero_files)
a549 = CellType("a549", a549_files)
hek_a = CellType("hek-a", hek_a_files)
vero_a = CellType("vero-a", vero_a_files)
a549_a = CellType("a549-a", a549_a_files)

# threshold all nans
hek.threshold_nans(inplace=True)
vero.threshold_nans(inplace=True)
a549.threshold_nans(inplace=True)
hek_a.threshold_nans(inplace=True)
vero_a.threshold_nans(inplace=True)
a549_a.threshold_nans(inplace=True)

# normalize datasets to reference set (a549._in)
hek.median_normalize(a549._in)
vero.median_normalize(a549._in)
a549.median_normalize(a549._in)
hek_a.median_normalize(a549._in)
vero_a.median_normalize(a549._in)
a549_a.median_normalize(a549._in)

In [ ]:
# generate .map files for use in superfold
for x in (vero, hek, a549):
    datasets = [x._in,
                 x._in_rep,
                 x._in_merged,
                 x._ex,
                 x._ex_rep,
                 x._ex_merged]
    
    for d in datasets:
        x.to_map_file(d.name, f'mapfiles/{d.name}_median-normalized-to-a549-in.map', data='med_normed')

In [ ]:
''' I have to define a standalone version of this for the a549/hek combined dataset (human)'''
def simple_to_map_file(data, fileout):

    stderr = data.med_normed_stderr
    profile = data.med_normed
    
    # file-ify into .map file
    with open(fileout, 'w') as f:
        # each line of the map file has the following structure:
        # num-->shape-->stderr-->nuc
        for i, shape in enumerate(profile):

            err = stderr[i]

            # adjust our nan values to be what .map files expect
            if np.isnan(shape):
                shape = '-999'
                err = '0'

            f.write(f"{data.num[i]}\t{shape}\t{str(err)}\t{data.seq[i]}\n")
            
human = SimpleNamespace()
human._ex = SimpleNamespace()

human._ex.profile = (a549._ex.profile + hek._ex_rep.profile)/2
human._ex.stderr = ((a549._ex.stderr**2 + hek._ex_rep.stderr**2)**.5)/2

human._ex.med_normed = human._ex.profile*(np.nanmedian(a549._in.profile)/np.nanmedian(human._ex.profile))
human._ex.med_normed_stderr = human._ex.stderr*(np.nanmedian(a549._in.profile)/np.nanmedian(human._ex.profile))

human._ex.num = hek._ex.num
human._ex.seq = hek._ex.seq

human._ex.name = "a549-ex-hek-ex-rep-merged"

human._ex.depth = hek._ex_rep.depth + a549._ex.depth

simple_to_map_file(human._ex, f'mapfiles/{human._ex.name}_median-normalized-to-a549-in.map')

In [ ]:
# compute pairing probabilities for each dataset
# commented lines are for deciding which are needed for the current analysis
from RNAtools import dotPlot

##### a549 #####

# TODO: add a549 replicate pairing probs to these
# a549._in.pprob = dotPlot("../structure_stuff/Superfold/results_a549-in_median-normalized-to-hek-in.map_44cc/merged_a549-in_median-normalized-to-hek-in.map_44cc.dp").pairingProb()
# a549._in_rep.pprob = dotPlot("../structure_stuff/Superfold/results_a549-in-replicate_median-normalized-to-hek-in.map_e2fd/merged_a549-in-replicate_median-normalized-to-hek-in.map_e2fd.dp").pairingProb()
# a549._ex.pprob = dotPlot("../structure_stuff/Superfold/results_a549-ex_median-normalized-to-hek-in.map_2bd9/merged_a549-ex_median-normalized-to-hek-in.map_2bd9.dp").pairingProb()
a549._in_merged.pprob = dotPlot("../structure_stuff/Superfold/results_a549-in-combined_median-normalized-to-a549-in.map_9329/merged_a549-in-combined_median-normalized-to-a549-in.map_9329.dp").pairingProb()

##### hek #####

# hek._in.pprob = dotPlot("../structure_stuff/Superfold/results_hek-in_median-normalized-to-hek-in.map_c7a5/merged_hek-in_median-normalized-to-hek-in.map_c7a5.dp").pairingProb()
# hek._in_rep.pprob = dotPlot("../structure_stuff/Superfold/results_hek-in-replicate_median-normalized-to-hek-in.map_6e57/merged_hek-in-replicate_median-normalized-to-hek-in.map_6e57.dp").pairingProb()
hek._in_merged.pprob = dotPlot("../structure_stuff/Superfold/results_hek-in-combined_median-normalized-to-a549-in.map_5d74/merged_hek-in-combined_median-normalized-to-a549-in.map_5d74.dp").pairingProb()

# hek._ex.pprob = dotPlot("../structure_stuff/Superfold/results_hek-ex_median-normalized-to-hek-in.map_ced9/merged_hek-ex_median-normalized-to-hek-in.map_ced9.dp").pairingProb()
# hek._ex_rep.pprob = dotPlot("../structure_stuff/Superfold/results_hek-ex-replicate_median-normalized-to-hek-in.map_2fd2/merged_hek-ex-replicate_median-normalized-to-hek-in.map_2fd2.dp").pairingProb()

# OOPS: There is no hek._ex_merged data right now lol
hek._ex_merged.pprob = dotPlot("../structure_stuff/Superfold/results_hek-ex-combined_median-normalized-to-a549-in.map_43ac/merged_hek-ex-combined_median-normalized-to-a549-in.map_43ac.dp").pairingProb()

##### vero #####

# vero._in.pprob = dotPlot("../structure_stuff/Superfold/results_vero-in_median-normalized-to-hek-in.map_b2f6/merged_vero-in_median-normalized-to-hek-in.map_b2f6.dp").pairingProb()
# vero._in_rep.pprob = dotPlot("../structure_stuff/Superfold/results_vero-in-replicate_median-normalized-to-hek-in.map_f219/merged_vero-in-replicate_median-normalized-to-hek-in.map_f219.dp").pairingProb()
vero._in_merged.pprob = dotPlot("../structure_stuff/Superfold/results_vero-in-combined_median-normalized-to-a549-in.map_cf4c/merged_vero-in-combined_median-normalized-to-a549-in.map_cf4c.dp").pairingProb()

# vero._ex.pprob = dotPlot("../structure_stuff/Superfold/results_vero-ex_median-normalized-to-hek-in.map_f205/merged_vero-ex_median-normalized-to-hek-in.map_f205.dp").pairingProb()
# vero._ex_rep.pprob = dotPlot("../structure_stuff/Superfold/results_vero-ex-replicate_median-normalized-to-hek-in.map_1bb2/merged_vero-ex-replicate_median-normalized-to-hek-in.map_1bb2.dp").pairingProb()
vero._ex_merged.pprob = dotPlot("../structure_stuff/Superfold/results_vero-ex-combined_median-normalized-to-a549-in.map_9f3c/merged_vero-ex-combined_median-normalized-to-a549-in.map_9f3c.dp").pairingProb()

##### human #####
human._ex.pprob = dotPlot("../structure_stuff/Superfold/results_a549-ex-hek-ex-rep-merged_median-normalized-to-a549-in.map_793a/merged_a549-ex-hek-ex-rep-merged_median-normalized-to-a549-in.map_793a.dp").pairingProb()

In [ ]:
''' creates a boolean list of paired vs unpaired for each pairing probability set, ambiguous positions are nan '''

datasets = (
    a549._in_merged,
    hek._in_merged,
#     hek._ex_merged.pprob,
    human._ex,
    vero._in_merged,
    vero._ex_merged
)

upperthresh = 0.95
lowerthresh = 0.05

# should I make a version of this in which ambiguous is a 0 instead? or maybe unpaired = -1 and paired = 1,
for data in datasets:
    probs = data.pprob
    data.paired = np.empty(len(probs))
    for i, p in enumerate(probs):
        if p >= upperthresh:
            data.paired[i] = True
        elif p <= lowerthresh:
            data.paired[i] = False
        else:
            data.paired[i] = np.nan
    
paired_list = []
unpaired_list = []

for paired, x in zip(a549._in_merged.paired, a549._in_merged.med_normed):
    if np.isnan(paired):
        continue
        
    if not paired:
        unpaired_list += [x]
    else:
        paired_list += [x]
        
print("unpaired shape avg:", round(np.nanmean(unpaired_list), 3))
print("paired shape avg:", round(np.nanmean(paired_list), 3))

In [ ]:
''' for each row in each table, get the absolute value of the difference in the relative reactivities '''

# p1 and p2 are profiles from shapemapper
def generate_reactivity_comp(seqs, p1, p2, absolute=False, errors=None):
    # Get whichever is the larger of the two sequences to base our comparison on
    seq_len = min(len(seqs[0]), len(seqs[1]))
    
    output = np.zeros(seq_len)
    
    for i in range(seq_len):
        # grab values
        v1 = p1[i]
        v2 = p2[i]

        # check if either value is NaN, if they are then ignore this row
        if v1 and v2:
            if absolute:
                output[i] = abs(v1 - v2)
            else:
                output[i] = v1 - v2
            
            
        else: output[i] = None
    
    # if errors were provided (as a tuple), generate a new set of propagated errors to return
    if errors:
        stderr = np.zeros(len(output))
        for i in range(len(stderr)):
            stderr[i] = sqrt(errors[0][i]**2 + errors[1][i]**2)
        return (output, stderr)
    else:    
        return output

In [ ]:
''' Inputs: profile, window size, and a nan threshold limit (as either an integer 0-100 or as a decimal 0-1). 
    Outputs: Altered profile with medians of the given window size'''

def generate_rolling_medians(data, window, nan_limit=.1):
    
    # failsafe for when I inevitably forget to make the percentage be a decimal and write it as an int
    if nan_limit > 1:
        nan_limit = nan_limit/100
    
    # nan_limit is the largest acceptable percentage of NaN values in the window, 25% by default
    
    output = np.full(len(data), np.nan)
    for i in range(window//2,len(data)-window//2):
        data_window = data[i-window//2:i+window//2]
        
        # if nan_limit is None, then just ignore NaNs and calculate medians based on all other values in the window
        if nan_limit is not None:
            output[i] = np.nanmedian(data_window)
        else:
            # count up the number of NaN values in this segment, divide by window to get a percentage, and compare to the limit
            if np.count_nonzero(np.isnan(data_window))/window > nan_limit:
                output[i] = np.nan
            else:
                output[i] = np.nanmedian(data_window)
    return output

# medians = generate_rolling_medians(reactivity_comp, window_size)

In [ ]:
''' Count mutations by type and evaluate the shape data at that location '''

# Transversions: A<->C, A<->T, G<->C, G<->T
# Transitions: A<->G, C<->T
# Insertions (relative to human): Human == '-', Vero != '-'
# Deletions (relative to human): Human != '-', Vero == '-'

def mutation_shapes(x, v, z):    
#     nums_human = list(x.num)
#     nums_vero = list(y.num)
    
    if ("vero" in x.name and "vero" not in v.name) or ("vero" not in x.name and "vero" in v.name):
#         print("aligning...")
        alignment = SeqIO.parse("clustalo-I20210622-162928-0115-39849737-p1m.clustal_num", 'clustal')

        aligned_seqs = []

        for record in alignment:
            aligned_seqs += [str(record.seq)]

        seqs = [np.array(list(aligned_seqs[0])), np.array(list(aligned_seqs[1]))]
                
                
        aligned_profiles = [list(x.med_normed.copy()), list(v.med_normed.copy()), list(z.med_normed.copy())]
        aligned_pairingprobs = [list(x.pprob.copy()), list(v.pprob.copy()), list(z.pprob.copy())]
        aligned_paired = [list(x.paired.copy()), list(v.paired.copy()), list(z.paired.copy())]
#         print("constructing aligned profiles...")
        for si, seq in enumerate(aligned_seqs):
            for i in range(len(seq)):
                if seq[i] == '-':
                    aligned_profiles[si].insert(i, np.nan)
                    aligned_pairingprobs[si].insert(i, np.nan)
                    aligned_paired[si].insert(i, np.nan)        
        # also do hek
        for i, base in enumerate(aligned_seqs[0]):
            if base == '-':
                aligned_profiles[2].insert(i, np.nan)
                aligned_pairingprobs[2].insert(i, np.nan)
                aligned_paired[2].insert(i, np.nan)
        
        # make sure that nums is the same length now as the aligned sequence
#         if (len(nums_human) != len(aligned_seqs[0])) or (len(nums_human) != len(aligned_seqs[0])):
#             raise Exception("length of nums does not match sequence length")
    else:
        # we should always be dealing with aligned data here, so throw an exception
        raise Exception("datasets are not human and vero")
    
    # shapes are stored here 
    mutation_shapes = {
        f"{x.name}": {
            "transitions": [],
            "transversions": [],
#             "insertions": [],
            "deletions": [],
            "non-mutated": [],
            "pairingprobs": aligned_pairingprobs[0],
            "paired": aligned_paired[0]
        },
        f"{z.name}": {
            "transitions": [],
            "transversions": [],
#             "insertions": [],
            "deletions": [],
            "non-mutated": [],
            "pairingprobs": aligned_pairingprobs[2],
            "paired": aligned_paired[0]
        },
        f"{v.name}": {
            "transitions": [],
            "transversions": [],
            "insertions": [],
            "non-mutated": [],
#             "deletions": []
            "pairingprobs": aligned_pairingprobs[1],
            "paired": aligned_paired[0]
        }
    }
    # NOTE: quick and dirty implementation of pairing probs instead of shape data 
    for (shape_a, shape_h, shape_v), (base_h, base_v) in zip((zip(aligned_profiles[0], aligned_profiles[2], aligned_profiles[1])),(zip(seqs[0], seqs[1]))):
#     for (shape_a, shape_h, shape_v), (base_h, base_v) in zip((zip(aligned_pairingprobs[0], aligned_pairingprobs[2], aligned_pairingprobs[1])),(zip(seqs[0], seqs[1]))):
        # if there is a mutation
        if (base_h != base_v):
            # Transversion
            if (base_h + base_v) in ('AC', 'AT', 'GC', 'GT', 'CA', 'TA', 'CG', 'TG'):
                mutation_shapes[z.name]['transversions'].append(shape_h)
                mutation_shapes[x.name]['transversions'].append(shape_a)
                mutation_shapes[v.name]['transversions'].append(shape_v)
                                                    
            # Transitions
            elif (base_h + base_v) in ('CT', 'TC', 'AG', 'GA'):
                mutation_shapes[z.name]['transitions'].append(shape_h)
                mutation_shapes[x.name]['transitions'].append(shape_a)
                mutation_shapes[v.name]['transitions'].append(shape_v)
                
            # Insertions
            elif base_h == '-' and base_v != '-':
#                 mutation_shapes['human']['insertions'].append(shape_h)
                mutation_shapes[v.name]['insertions'].append(shape_v)
                
            # Deletions
            elif base_h != '-' and base_v == '-':
                mutation_shapes[z.name]['deletions'].append(shape_h)
                mutation_shapes[x.name]['deletions'].append(shape_a)
#                 mutation_shapes['vero']['deletions'].append(shape_v)

        else:
            mutation_shapes[z.name]['non-mutated'].append(shape_h)
            mutation_shapes[x.name]['non-mutated'].append(shape_a)
            mutation_shapes[v.name]['non-mutated'].append(shape_v)
                
    return mutation_shapes
#         temp = [None]*2
#         temp[0] = aligned_seqs[1]
#         temp[1] = aligned_seqs[0]
#         aligned_seqs = temp

            # now our profiles should match with our sequences, num needs to be updated
    #         aligned_nums = [list(range(len(aligned_seqs[0]))), list(range(len(aligned_seqs[1])))]

    #         nums = aligned_nums

    #         diffs = np.array(abs(data[0].med_normed - data[1].med_normed))

In [ ]:
x = a549._in_merged
v = vero._in_merged
z = hek._in_merged

# x = human._ex
# v = vero._ex_merged
# z = hek._ex_merged




shapes = mutation_shapes(x, v, z)

print('Shape values by mutation type (relative to human)- Mean, Standard Deviation, Median')
print(f'{x.name} \tMean\tStd\tMedian')
print('\tTransitions:\t' + str(round(np.nanmean(shapes[x.name]['transitions']), 3)) + '\t' + str(round(np.nanstd(shapes[x.name]['transitions']), 3)) + '\t' + str(round(np.nanmedian(shapes[x.name]['transitions']), 3)))
print('\tTransversions:\t' + str(round(np.nanmean(shapes[x.name]['transversions']), 3)) + '\t' + str(round(np.nanstd(shapes[x.name]['transversions']), 3)) + '\t' + str(round(np.nanmedian(shapes[x.name]['transversions']), 3)))
print('\tDeletions:\t' + str(round(np.nanmean(shapes[x.name]['deletions']), 3)) + '\t' + str(round(np.nanstd(shapes[x.name]['deletions']), 3)) + '\t' + str(round(np.nanmedian(shapes[x.name]['deletions']), 3)))

print(f'\n{v.name} \tMean\tStd\tMedian')
print('\tTransitions:\t' + str(round(np.nanmean(shapes[v.name]['transitions']), 3)) + '\t' + str(round(np.nanstd(shapes[v.name]['transitions']), 3)) + '\t' + str(round(np.nanmedian(shapes[v.name]['transitions']), 3)))
print('\tTransversions:\t' + str(round(np.nanmean(shapes[v.name]['transversions']), 3)) + '\t' + str(round(np.nanstd(shapes[v.name]['transversions']), 3)) + '\t' + str(round(np.nanmedian(shapes[v.name]['transversions']), 3)))
print('\tInsertions:\t' + str(round(np.nanmean(shapes[v.name]['insertions']), 3)) + '\t' + str(round(np.nanstd(shapes[v.name]['insertions']), 3)) + '\t' + str(round(np.nanmedian(shapes[v.name]['insertions']), 3)))

t_transitions = stats.ttest_ind(shapes[x.name]['transitions'], shapes[v.name]['transitions'], nan_policy='omit')
t_transversions = stats.ttest_ind(shapes[x.name]['transversions'], shapes[v.name]['transversions'], nan_policy='omit')
t_human = stats.ttest_ind(shapes[x.name]['transversions'], shapes[x.name]['transitions'], nan_policy='omit')
t_vero = stats.ttest_ind(shapes[v.name]['transversions'], shapes[v.name]['transitions'], nan_policy='omit')

print(f'\nt-tests ({x.name} vs {v.name})')
print('\tTransitions:\t t = ' + str(round(t_transitions[0], 5)) + '\t p = ' + str(round(t_transitions[1], 5)))
print('\tTransversions:\t t = ' + str(round(t_transversions[0], 5)) + '\t p = ' + str(round(t_transversions[1], 5)))
print('\tHuman:\t t = ' + str(round(t_human[0], 5)) + '\t p = ' + str(round(t_human[1], 5)))
print('\tVero:\t t = ' + str(round(t_vero[0], 5)) + '\t p = ' + str(round(t_vero[1], 5)))
# print('\tInsertions:\t' + str(stats.ttest_ind(shapes['human']['insertions'], shapes['vero']['insertions'])))

In [ ]:
''' count mutation types '''
    
print(x.name)
print(f'\tTransitions\t' + str(len(shapes[x.name]['transitions'])))
print(f'\tTransversions\t' + str(len(shapes[x.name]['transversions'])))
print(f'\tDeletions\t' + str(len(shapes[x.name]['deletions'])))
print()

print(z.name)
print(f'\tTransitions:\t' + str(len(shapes[z.name]['transitions'])))
print(f'\tTransversions:\t' + str(len(shapes[z.name]['transversions'])))
print(f'\tDeletions:\t' + str(len(shapes[z.name]['deletions'])))
print()

print(v.name)
print(f'\tTransitions:\t' + str(len(shapes[v.name]['transitions'])))
print(f'\tTransversions:\t' + str(len(shapes[v.name]['transversions'])))
print(f'\tInsertions:\t' + str(len(shapes[v.name]['insertions'])))



In [ ]:
''' generate violin box strip plots for the mutation types '''

%matplotlib inline

import seaborn as sns

non_h = np.array(shapes[z.name]['non-mutated'])
non_a = np.array(shapes[x.name]['non-mutated'])
# non_a = np.array(shapes['a549']['non-mutated'])
non_v = np.array(shapes[v.name]['non-mutated'])

mut_h = np.array(shapes[z.name]['transitions'] + shapes[z.name]['transversions'])
mut_a = np.array(shapes[x.name]['transitions'] + shapes[x.name]['transversions'])
mut_v = np.array(shapes[v.name]['transitions'] + shapes[v.name]['transversions'])

tsi_h = np.array(shapes[z.name]['transitions'])
tvr_h = np.array(shapes[z.name]['transversions'])

tsi_a = np.array(shapes[x.name]['transitions'])
tvr_a = np.array(shapes[x.name]['transversions'])

tsi_v = np.array(shapes[v.name]['transitions'])
tvr_v = np.array(shapes[v.name]['transversions'])

del_h = np.array(shapes[z.name]['deletions'])
del_a = np.array(shapes[x.name]['deletions'])
ins_v = np.array(shapes[v.name]['insertions'])

allmut_h = np.array(shapes[z.name]['transitions'] + shapes[z.name]['transversions'] + shapes[z.name]['deletions'])
allmut_a = np.array(shapes[x.name]['transitions'] + shapes[x.name]['transversions'] + shapes[x.name]['deletions'])
allmut_v = np.array(shapes[v.name]['transitions'] + shapes[v.name]['transversions'] + shapes[v.name]['insertions'])


# kill all the nan values
non_h = non_h[~np.isnan(non_h)]
mut_h = mut_h[~np.isnan(mut_h)]
del_h = del_h[~np.isnan(del_h)]
tsi_h = tsi_h[~np.isnan(tsi_h)]
tvr_h = tvr_h[~np.isnan(tvr_h)]
allmut_h = allmut_h[~np.isnan(allmut_h)]

non_a = non_a[~np.isnan(non_a)]
mut_a = mut_a[~np.isnan(mut_a)]
del_a = del_a[~np.isnan(del_a)]
tsi_a = tsi_a[~np.isnan(tsi_a)]
tvr_a = tvr_a[~np.isnan(tvr_a)]
allmut_a = allmut_a[~np.isnan(allmut_a)]

non_v = non_v[~np.isnan(non_v)]
mut_v = mut_v[~np.isnan(mut_v)]
ins_v = ins_v[~np.isnan(ins_v)]
tsi_v = tsi_v[~np.isnan(tsi_v)]
tvr_v = tvr_v[~np.isnan(tvr_v)]
allmut_v = allmut_v[~np.isnan(allmut_v)]
    
# mean_non_h = np.mean(non_h)
# mean_non_v = np.mean(non_v)
# mean_mut_h = np.mean(mut_h)
# mean_mut_v = np.mean(mut_v)

######## plotting stuff ########

left_inches = 0.9
right_inches = 0.4
fig_width = 22.5
fig_height = 10
num_panels = 3

left_percent = left_inches/fig_width
right_percent = 1-right_inches/fig_width

fig = plt.figure(figsize=(fig_width,fig_height))

#     create axes for subplots
axes = []    
for i in range(num_panels):
    axes += [plt.subplot(1, num_panels, i + 1)]

for ax in axes:
    ax.set_ylim(-.1, 1.2)
    
# plt.subplots_adjust(hspace=0.5, left=left_percent,right=right_percent, top=0.98, bottom=0.02)

# axes[0].boxplot((ins_v, del_h, mut_v, mut_h, non_v, non_h), vert=False, labels=("vero-in inserted", "a549-in deleted", "vero-in mutated", "a549-in mutated", "vero-in non-mutated", "a549-in non-mutated"))

# t_non = stats.ttest_ind(non_h, non_v)
# t_mut = stats.ttest_ind(mut_h, mut_v)
# t_indel = stats.ttest_ind(del_h, ins_v)
# t_a549_non_vs_mut = stats.ttest_ind(non_h, mut_h)
# t_vero_non_vs_mut = stats.ttest_ind(non_v, mut_v)
# t_a549_non_vs_del = stats.ttest_ind(non_h, del_h)
# t_vero_non_vs_ins = stats.ttest_ind(non_v, ins_v)
# t_a549_del_vs_mut = stats.ttest_ind(del_h, mut_h)
# t_vero_ins_vs_mut = stats.ttest_ind(ins_v, mut_v)

# axes[0].boxplot((non_h, mut_h, del_h, tsi_h, tvr_h), labels=("non-mutated", "mutated", "deleted", "transition", "transversion"))
# axes[0].boxplot((non_a, mut_a), labels=("non-mutated", "mutated"))
# plt.sca(axes[0])
# sns.violinplot(x=["non-mutated", "mutated"], y=(non_a, mut_a))

# axes[0].violinplot((non_a, mut_a))

point_size = 2
point_alpha = .1
jitter = .25
box_width = .15
cut = 0
gridsize = 500

sns.violinplot(ax=axes[0], data=[non_a, allmut_a], rasterized=True, inner=None, color='lightgray', zorder=0, cut=cut, gridsize=gridsize)
sns.stripplot(ax=axes[0], data=[non_a, allmut_a], alpha=point_alpha, rasterized=True, size=point_size, color='black', jitter = jitter, zorder=1)
sns.boxplot(ax=axes[0], data=[non_a, allmut_a], width=box_width, zorder=2, showfliers=False, boxprops={'zorder': 2}, color='.6')
# axes[0].set_title("a549-in-combined")
axes[0].set_title(x.name)
# axes[0].set(xticklabels=('non-mutated', 'transition/transversion'))
# axes[0].set(xticklabels=('non-mutated', 'deletion'))
# axes[0].set(xticklabels=('transition/transversion', 'deletion'))
axes[0].set(xticklabels=('non-mutated', 'mutated (any kind)'))
# statistical annotation
y, h = 1.1, .02
axes[0].plot([0, 0, 1, 1], [y, y+h, y+h, y], lw=1.5, c='black')
axes[0].text(.5, y+h, f"p = {round(t_a549_non_vs_allmut[1], 3)}", ha='center', va='bottom', color='black')

sns.violinplot(ax=axes[1], data=[non_h, allmut_h], rasterized=True, inner=None, color='lightgray', zorder=0, cut=cut, gridsize=gridsize)
sns.stripplot(ax=axes[1], data=[non_h, allmut_h], alpha=point_alpha, rasterized=True, size=point_size, color='black', jitter = jitter, zorder=1)
sns.boxplot(ax=axes[1], data=[non_h, allmut_h], width=box_width, zorder=2, showfliers=False, boxprops={'zorder': 2}, color='.6')
axes[1].set_title(z.name)
# axes[1].set(xticklabels=('non-mutated', 'transition/transversion'))
# axes[1].set(xticklabels=('non-mutated', 'deletion'))
# axes[1].set(xticklabels=('transition/transversion', 'deletion'))
axes[1].set(xticklabels=('non-mutated', 'mutated (any kind)'))
# statistical annotation
y, h = 1.1, .02
axes[1].plot([0, 0, 1, 1], [y, y+h, y+h, y], lw=1.5, c='black')
axes[1].text(.5, y+h, f"p = {round(t_hek_non_vs_allmut[1], 3)}", ha='center', va='bottom', color='black')

sns.violinplot(ax=axes[2], data=[non_v, allmut_v], rasterized=True, inner=None, color='lightgray', zorder=0, cut=cut, gridsize=gridsize)
sns.stripplot(ax=axes[2], data=[non_v, allmut_v], alpha=point_alpha, rasterized=True, size=point_size, color='black', jitter = jitter, zorder=1)
sns.boxplot(ax=axes[2], data=[non_v, allmut_v], width=box_width, zorder=2, showfliers=False, boxprops={'zorder': 2}, color='.6')
axes[2].set_title(v.name)
# axes[2].set(xticklabels=('non-mutated', 'transition/transversion'))
# axes[2].set(xticklabels=('non-mutated', 'insertion'))
# axes[2].set(xticklabels=('transition/transversion', 'insertion'))
axes[2].set(xticklabels=('non-mutated', 'mutated (any kind)'))

# statistical annotation
y, h = 1.1, .02
axes[2].plot([0, 0, 1, 1], [y, y+h, y+h, y], lw=1.5, c='black')
axes[2].text(.5, y+h, f"p = {round(t_vero_non_vs_allmut[1], 3)}", ha='center', va='bottom', color='black')

# axes[1].boxplot((non_v, mut_v, ins_v, tsi_v, tvr_v), labels=("non-mutated", "mutated", "inserted", "transition", "transversion"))
# axes[1].set_title("vero-in-combined")

plt.savefig(f"plots_boxplots/mutation_analysis_ex-vivo_rasterized_alpha020_withviolin_nonvsallmuts.pdf")

In [ ]:
''' Generate all of the t tests for the above cells '''

t_non = stats.ttest_ind(non_h, non_v)
t_mut = stats.ttest_ind(mut_h, mut_v)
t_indel = stats.ttest_ind(del_h, ins_v)
t_tsi = stats.ttest_ind(tsi_h, tsi_v)
t_tvr = stats.ttest_ind(tvr_h, tvr_v)

t_a549_non_vs_mut = stats.ttest_ind(non_a, mut_a)
t_a549_non_vs_del = stats.ttest_ind(non_a, del_a)
t_a549_non_vs_tsi = stats.ttest_ind(non_a, tsi_a)
t_a549_non_vs_tvr = stats.ttest_ind(non_a, tsi_a)
t_a549_del_vs_mut = stats.ttest_ind(del_a, mut_a)
t_a549_tsi_vs_tvr = stats.ttest_ind(tsi_a, tvr_a)
t_a549_non_vs_allmut = stats.ttest_ind(non_a, allmut_a)

t_vero_non_vs_mut = stats.ttest_ind(non_v, mut_v)
t_vero_non_vs_ins = stats.ttest_ind(non_v, ins_v)
t_vero_non_vs_tsi = stats.ttest_ind(non_v, tsi_v)
t_vero_non_vs_tvr = stats.ttest_ind(non_v, tsi_v)
t_vero_ins_vs_mut = stats.ttest_ind(ins_v, mut_v)
t_vero_tsi_vs_tvr = stats.ttest_ind(tsi_v, tvr_v)
t_vero_non_vs_allmut = stats.ttest_ind(non_v, allmut_v)

t_hek_non_vs_mut = stats.ttest_ind(non_h, mut_h)
t_hek_non_vs_del = stats.ttest_ind(non_h, del_h)
t_hek_non_vs_tsi = stats.ttest_ind(non_h, tsi_h)
t_hek_non_vs_tvr = stats.ttest_ind(non_h, tsi_h)
t_hek_del_vs_mut = stats.ttest_ind(del_h, mut_h)
t_hek_tsi_vs_tvr = stats.ttest_ind(tsi_h, tvr_h)
t_hek_non_vs_allmut = stats.ttest_ind(non_h, allmut_h)


kw_a549_non_vs_mut = stats.kruskal(non_a, mut_a, nan_policy="omit")
kw_hek_non_vs_mut = stats.kruskal(non_h, mut_h, nan_policy="omit")
kw_vero_non_vs_mut = stats.kruskal(non_v, mut_v, nan_policy="omit")

ks_a549_non_vs_mut = stats.kstest(non_a, mut_a)
ks_hek_non_vs_mut = stats.kstest(non_h, mut_h)
ks_vero_non_vs_mut = stats.kstest(non_v, mut_v)

print("----- T-TESTS -----")
print("a549-in")
print("\tNon vs Mut:\t t = " + str(round(t_a549_non_vs_mut[0], 5)) + "\t p = " + str(round(t_a549_non_vs_mut[1], 5)))
print("\tNon vs Del:\t t = " + str(round(t_a549_non_vs_del[0], 5)) + "\t p = " + str(round(t_a549_non_vs_del[1], 5)))
print("\tNon vs Tsi:\t t = " + str(round(t_a549_non_vs_tsi[0], 5)) + "\t p = " + str(round(t_a549_non_vs_tsi[1], 5)))
print("\tNon vs Tvr:\t t = " + str(round(t_a549_non_vs_tvr[0], 5)) + "\t p = " + str(round(t_a549_non_vs_tvr[1], 5)))
print("\tDel vs Mut:\t t = " + str(round(t_a549_del_vs_mut[0], 5)) + "\t p = " + str(round(t_a549_del_vs_mut[1], 5)))
print("\tTsi vs Tvr:\t t = " + str(round(t_a549_tsi_vs_tvr[0], 5)) + "\t p = " + str(round(t_a549_tsi_vs_tvr[1], 5)))
print("\nhek-in")
print("\tNon vs Mut:\t t = " + str(round(t_hek_non_vs_mut[0], 5)) + "\t p = " + str(round(t_hek_non_vs_mut[1], 5)))
print("\tNon vs Del:\t t = " + str(round(t_hek_non_vs_del[0], 5)) + "\t p = " + str(round(t_hek_non_vs_del[1], 5)))
print("\tNon vs Tsi:\t t = " + str(round(t_hek_non_vs_tsi[0], 5)) + "\t p = " + str(round(t_hek_non_vs_tsi[1], 5)))
print("\tNon vs Tvr:\t t = " + str(round(t_hek_non_vs_tvr[0], 5)) + "\t p = " + str(round(t_hek_non_vs_tvr[1], 5)))
print("\tDel vs Mut:\t t = " + str(round(t_hek_del_vs_mut[0], 5)) + "\t p = " + str(round(t_hek_del_vs_mut[1], 5)))
print("\tTsi vs Tvr:\t t = " + str(round(t_hek_tsi_vs_tvr[0], 5)) + "\t p = " + str(round(t_hek_tsi_vs_tvr[1], 5)))
print("\nvero-in")
print("\tNon vs Mut:\t t = " + str(round(t_vero_non_vs_mut[0], 5)) + "\t p = " + str(round(t_vero_non_vs_mut[1], 5)))
print("\tNon vs Ins:\t t = " + str(round(t_vero_non_vs_ins[0], 5)) + "\t p = " + str(round(t_vero_non_vs_ins[1], 5)))
print("\tNon vs Tsi:\t t = " + str(round(t_vero_non_vs_tsi[0], 5)) + "\t p = " + str(round(t_vero_non_vs_tsi[1], 5)))
print("\tNon vs Tvr:\t t = " + str(round(t_vero_non_vs_tvr[0], 5)) + "\t p = " + str(round(t_vero_non_vs_tvr[1], 5)))
print("\tIns vs Mut:\t t = " + str(round(t_vero_ins_vs_mut[0], 5)) + "\t p = " + str(round(t_vero_ins_vs_mut[1], 5)))
print("\tTsi vs Tvr:\t t = " + str(round(t_vero_tsi_vs_tvr[0], 5)) + "\t p = " + str(round(t_vero_tsi_vs_tvr[1], 5)))
print("\na549-in vs vero-in")
print("\tNon-mutated:\t t = " + str(round(t_non[0], 5)) + "\t p = " + str(round(t_non[1], 5)))
print("\tMutated:\t t = " + str(round(t_mut[0], 5)) + "\t p = " + str(round(t_mut[1], 5)))
print("\tIndel:  \t t = " + str(round(t_indel[0], 5)) + "\t p = " + str(round(t_indel[1], 5)))
print("\tTransition:  \t t = " + str(round(t_tvr[0], 5)) + "\t p = " + str(round(t_tvr[1], 5)))
print("\tTransversion:  \t t = " + str(round(t_tsi[0], 5)) + "\t p = " + str(round(t_tsi[1], 5)))

print("\n\n----- KRUSKAL-WALLIS -----")
print("\ta549:\t\t " "H = " + str(round(kw_a549_non_vs_mut[0], 3)), "\t p = " + str(round(kw_a549_non_vs_mut[1], 3)))
print("\thek:\t\t " "H = " + str(round(kw_hek_non_vs_mut[0], 3)), "\t p = " + str(round(kw_hek_non_vs_mut[1], 3)))
print("\tvero:\t\t " "H = " + str(round(kw_vero_non_vs_mut[0], 3)), "\t p = " + str(round(kw_vero_non_vs_mut[1], 3)))

print("\n\n----- K-S -----")
print("\ta549:\t\t " "H = " + str(round(ks_a549_non_vs_mut[0], 3)), "\t p = " + str(round(ks_a549_non_vs_mut[1], 3)))
print("\thek:\t\t " "H = " + str(round(ks_hek_non_vs_mut[0], 3)), "\t p = " + str(round(ks_hek_non_vs_mut[1], 3)))
print("\tvero:\t\t " "H = " + str(round(ks_vero_non_vs_mut[0], 3)), "\t p = " + str(round(ks_vero_non_vs_mut[1], 3)))

In [ ]:
''' make x/y scatter plots ('Belgian plots') with weeks lab coloring based on shape '''

%matplotlib inline
from RNAtools import dotPlot
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")


comparisons = (
#     (hek._in, hek._in_rep),
#     (hek._ex, hek._ex_rep),
#     (vero._in, vero._in_rep),
#     (vero._ex, vero._ex_rep),
#     (a549._in_merged, vero._in_merged),
#     (vero._in_merged, a549._in_merged),
#     (human._ex, vero._ex_merged),
#     (a549._in, a549._in_rep),
#     (vero._ex_merged, human._ex),    
#     (a549._in, a549._in_rep),
#     (hek._ex_rep, a549._ex),
#     (hek._in_merged, a549._in_merged),
#     (a549._in_merged, human._ex),
#     (human._ex, a549._in_merged),
#     (a549._in_rep, hek._in_rep),
#     (a549._ex, hek._ex),
#     (a549._ex, hek._ex_rep)
#     (human._ex, a549._in_merged),
#     (vero._ex_merged, vero._in_merged),
#     (vero._in_merged, vero._ex_merged),
#     (hek._in_merged, human._ex),
    (human._ex, hek._in_merged),
)

def belgian_scatter(comparison, scramble=False):
    left_inches = 0.9
    right_inches = 0.4

    fig_width = 8.5
    fig_height = 11
    # fig_width = 10
    # fig_width = max(7,sp_width+left_inches+right_inches)
    num_panels = 1
    
#     fig_height = 10*(math.ceil(num_panels/2))
    # fig_height = 10*5
    # fig_height = 1*num_panels-1  #heatbar height
    # fig_height = 300

#     left_percent = left_inches/fig_width
#     right_percent = 1-right_inches/fig_width

    fig = plt.figure(figsize=(fig_width,fig_height))

    #     create axes for subplots
#     axes = []    
#     axes += [plt.subplot(num_panels, 1, i + 1)]
    ax = plt.subplot(1, 1, 1)
    # for i in range(len(comparisons)):
    #     axes += [plt.subplot(num_panels, 1, i + 1)]

#     plt.subplots_adjust(hspace=0.5, left=left_percent,right=right_percent)
    
#     y, x = comparison
    x, y = comparison

#     fx = x.profile.copy()
#     fy = y.profile.copy()

    if ("vero" in x.name and "vero" not in y.name) or ("vero" not in x.name and "vero" in y.name):
        print("aligning...")
        alignment = SeqIO.parse("clustalo-I20210622-162928-0115-39849737-p1m.clustal_num", 'clustal')

        aligned_seqs = []

        for record in alignment:
            aligned_seqs += [str(record.seq)]
    
#         temp = [None]*2
#         temp[0] = aligned_seqs[1]
#         temp[1] = aligned_seqs[0]
#         aligned_seqs = temp
        
        # lets make some specific aligned variables
        aligned_profiles = [list(x.med_normed.copy()), list(y.med_normed.copy())]

        
        print("constructing aligned profiles...")
        for si, seq in enumerate(aligned_seqs):
            for i in range(len(seq)):
                if seq[i] == '-':
                    aligned_profiles[si].insert(i, np.nan)

        # now our profiles should match with our sequences, num needs to be updated
#         aligned_nums = [list(range(len(aligned_seqs[0]))), list(range(len(aligned_seqs[1])))]

#         nums = aligned_nums
#         seqs = [np.array(list(aligned_seqs[0])), np.array(list(aligned_seqs[1]))]
#         diffs = np.array(abs(data[0].med_normed - data[1].med_normed))
        
        fx, fy = aligned_profiles
        
        fx = np.array(fx)
        fy = np.array(fy)
        
    else:
        # real
#         diffs = np.array(data[0].med_normed - data[1].med_normed)
        # absolute
        fx = x.med_normed.copy()
        fy = y.med_normed.copy()

    # delete all nan values (matched, of course) from the datasets
#     nan_mask = np.isnan(fx) | np.isnan(fy)
#     fx = np.delete(fx, nan_mask)
#     fy = np.delete(fy, nan_mask)

#     median normalize to our reference (a549._in.profile's mean)
#     fx = fx*(np.nanmedian(a549._in.profile)/np.nanmedian(fx))
#     fy = fy*(np.nanmedian(a549._in.profile)/np.nanmedian(fy))

    # "Alternatively we can also color by BP probability i.e BP 0-0.1 black, BP = 0.1-0.9 gray, and BP=0.9-1 red."
#     pprob = np.array(dotPlot("../structure_stuff/Superfold/results_a549-ex-hek-ex-rep-merged_median-normalized-to-a549-in.map_793a/merged_a549-ex-hek-ex-rep-merged_median-normalized-to-a549-in.map_793a.dp").pairingProb())

#     black_mask = 0.1 > pprob
#     yellow_mask = (0.1 <= pprob) & (0.8 > pprob)
#     orange_mask = (0.8 <= pprob) & (0.9 > pprob)
#     red_mask = 0.9 <= pprob

    # square-in-a-square masks
#     grey_mask = (0 > fx) | (0 > fy)
#     black_mask = ((0 <= fx) & (0.4 > fx)) | ((0 <= fy) & (0.4 > fy))
#     orange_mask = ((0.4 <= fx) & (0.85 > fx)) | ((0.4 <= fy) & (0.85 > fy))
#     red_mask = (0.85 <= fx) & (0.85 <= fy)
#     black_mask[grey_mask] = False
#     orange_mask[black_mask] = False
#     orange_mask[grey_mask] = False
    
    if scramble:
        np.random.shuffle(fx)
    
    grey_mask = (0 > fx)
    black_mask = (0 <= fx) & (0.4 > fx)
    orange_mask = (0.4 <= fx) & (0.85 > fx)
    red_mask = (0.85 <= fx)


    
    
#     z_paired = np.polyfit(fx_paired, fy_paired, 1)
#     p_paired = np.poly1d(z_paired)

#     z_unpaired = np.polyfit(fx_unpaired, fy_unpaired, 1)
#     p_unpaired = np.poly1d(z_unpaired)


#     print(f"name: {x.name}, {y.name};  \t slope: {round(z[0], 3)}; medians: {round(np.nanmedian(fx), 3)}, {round(np.nanmedian(fy), 3)}")


#     ax.plot(range(-1,6),range(-1,6),"r--", lw=1)    

#     scatter = ax.scatter(fx, fy, alpha=.05, color='black', rasterized=True)    

#     scatter_grey = ax.scatter(fx[grey_mask], fy[grey_mask], alpha=.5, color='grey', rasterized=True)


#     scatter_red = ax.scatter(fx[red_mask], fy[red_mask], alpha=.15, color='red', rasterized=True)
#     scatter_orange = ax.scatter(fx[orange_mask], fy[orange_mask], alpha=.15, color='orange', rasterized=True)
#     scatter_grey = ax.scatter(fx[grey_mask], fy[grey_mask], alpha=.15, color='grey', rasterized=True)
#     scatter_black = ax.scatter(fx[black_mask], fy[black_mask], alpha=.15, color=".2", rasterized=True)                 



    scatter_red = ax.scatter(fx[red_mask], fy[red_mask], alpha=.15, color='red', rasterized=True)
    scatter_orange = ax.scatter(fx[orange_mask], fy[orange_mask], alpha=.15, color='orange', rasterized=True)
    scatter_grey = ax.scatter(fx[grey_mask], fy[grey_mask], alpha=.15, color='grey', rasterized=True)
    scatter_black = ax.scatter(fx[black_mask], fy[black_mask], alpha=.15, color=".2", rasterized=True)                 
#     scatter_unpaired_both = ax.scatter(fx_unpaired, fy_unpaired, alpha=1, color='red', rasterized=True)
#     scatter_paired_both = ax.scatter(fx_paired, fy_paired, alpha=1, color='black', rasterized=True)


#     scatter_paired_one = ax.scatter(x.med_normed[paired_one], y.med_normed[paired_one], alpha=1, color='orange', rasterized=True)

    # delete all nan values (matched, of course) from the datasets because polyfit can't handle nans
    nan_mask = np.isnan(fx) | np.isnan(fy)
    fx = np.delete(fx, nan_mask)
    fy = np.delete(fy, nan_mask)

    # re-define masks according to new (no nans) dimensions
    
# #     original (Belgian) masks
    grey_mask = (0 > fx)
    black_mask = (0 <= fx) & (0.4 > fx)
    orange_mask = (0.4 <= fx) & (0.85 > fx)
    red_mask = (0.85 <= fx)
    
    # square-in-a-square masks
#     grey_mask = (0 > fx) | (0 > fy)
#     black_mask = ((0 <= fx) & (0.4 > fx)) | ((0 <= fy) & (0.4 > fy))
#     orange_mask = ((0.4 <= fx) & (0.85 > fx)) | ((0.4 <= fy) & (0.85 > fy))
#     red_mask = (0.85 <= fx) & (0.85 <= fy)

#     black_mask[grey_mask] = False
#     orange_mask[black_mask] = False
#     orange_mask[grey_mask] = False
    
    grey = [fx[grey_mask], fy[grey_mask]]
    black = [fx[black_mask], fy[black_mask]]
    orange = [fx[orange_mask], fy[orange_mask]]
    red = [fx[red_mask], fy[red_mask]]

#     plt.plot(x, y, 'bo')
#     plt.plot(x, a*x, 'r-')
#     plt.show()

#     z = np.polyfit(grey[0], grey[1], 1)
#     p = np.poly1d(z)
#     ax.plot(grey[0], p(grey[0]),"grey", lw=3)
#     text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(grey[1],p(grey[0])):0.3f}$"
#     ax.text(0.05, 0.95, text,transform=ax.transAxes,
#                fontsize=14, verticalalignment='top', color='grey')


    lx = fx[:,np.newaxis]
    a, lx_res, _, _ = np.linalg.lstsq(lx, fy)
#     ax.plot(lx, a*lx,"--k", lw=3)
    text = f"$y={a[0]:0.3f}x$\n$R^2 = {r2_score(fy,a*fx):0.3f}$"
    ax.text(0.25, 0.95, text,transform=ax.transAxes,
               fontsize=14, verticalalignment='top', color='green')

    black[0] = black[0][:,np.newaxis]
    a, k_res, _, _ = np.linalg.lstsq(black[0], black[1])
#     z = np.polyfit(black[0], black[1], 1)
#     p = np.poly1d(z)
    ax.plot(black[0], a*black[0],"black", lw=3)
    text = f"$y={a[0]:0.3f}x$\n$R^2 = {r2_score(black[1],a*black[0]):0.3f}$"
    ax.text(0.05, 0.95, text,transform=ax.transAxes,
               fontsize=14, verticalalignment='top', color='.2')

    orange[0] = orange[0][:,np.newaxis]
    a, o_res, _, _ = np.linalg.lstsq(orange[0], orange[1])
#     z = np.polyfit(orange[0], orange[1], 1)
#     p = np.poly1d(z)
    ax.plot(orange[0], a*orange[0],"yellow", lw=3)
    text = f"$y={a[0]:0.3f}x$\n$R^2 = {r2_score(orange[1],a*orange[0]):0.3f}$"
    ax.text(0.05, 0.85, text,transform=ax.transAxes,
               fontsize=14, verticalalignment='top', color='orange')

    red[0] = red[0][:,np.newaxis]
    a, r_res, _, _ = np.linalg.lstsq(red[0], red[1])
#     z = np.polyfit(red[0], red[1], 1)
#     p = np.poly1d(z)
    ax.plot(red[0], a*red[0],"firebrick", lw=3)
    text = f"$y={a[0]:0.3f}x$\n$R^2 = {r2_score(red[1],a*red[0]):0.3f}$"
    ax.text(0.05, 0.75, text,transform=ax.transAxes,
               fontsize=14, verticalalignment='top', color='red')

#     ax.plot(fx_paired, p_paired(fx_paired),"b--", lw=1)
#     ax.plot(fx_unpaired, p_unpaired(fx_unpaired),"r--", lw=1)

    ax.set_ylim(-.5, 4)
    ax.set_xlim(-.5, 4)
#     ax.set_title(f"{x.name} vs {y.name}")

    ax.set_xlabel(x.name)
    ax.set_ylabel(y.name)

    if scramble:
        ax.set_xlabel(f"scrambled {x.name}")
    
    ax.set_aspect(1)
    
    print(k_res/len(black), o_res/len(orange), r_res/len(red), lx_res/len(lx))

#     text = f"$y={z_paired[0]:0.3f}\;x{z_paired[1]:+0.3f}$\n$R^2 = {r2_score(fy_paired,p_paired(fx_paired)):0.3f}$"
# #     text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(fy,p(fx)):0.3f}$"
# #     text = f"$R^2 = {r2_score(fy,p(fx)):0.3f}$"
#     ax.text(0.05, 0.95, text,transform=ax.transAxes,
#                    fontsize=14, verticalalignment='top', color='blue')

#     text = f"$y={z_unpaired[0]:0.3f}\;x{z_unpaired[1]:+0.3f}$\n$R^2 = {r2_score(fy_unpaired,p_unpaired(fx_unpaired)):0.3f}$"

#     ax.text(0.05, 0.85, text,transform=ax.transAxes,
#                    fontsize=14, verticalalignment='top', color='red')

    plt.savefig(f"plots_scatters/{x.name}_vs_{y.name}_shape_scatters_Belgium_linestozero_withtotal.pdf")
    

for c in comparisons:
    belgian_scatter(c, scramble=False)

In [ ]:
''' make the shape skyline plots of the two replicates '''
%matplotlib inline

comparisons = (
#     (hek._in, hek._in_rep),
#     (hek._ex, hek._ex_rep),
#     (vero._in, vero._in_rep),
#     (vero._ex, vero._ex_rep),
#     (human._ex, a549._in_merged),
#     (a549._ex, hek._ex),
#     (a549._ex, hek._ex_rep),
    (vero._in, vero._in_rep),
)


left_inches = 0.9
right_inches = 0.4
fig_width = 20

num_panels = len(comparisons)

fig_height = 10*num_panels

left_percent = left_inches/fig_width
right_percent = 1-right_inches/fig_width

fig = plt.figure(figsize=(fig_width,fig_height), facecolor='white')

#     create axes for subplots
axes = []    
for i in range(len(comparisons)):
    axes += [plt.subplot(num_panels, 1, i + 1)]

plt.subplots_adjust(hspace=0.5, left=left_percent,right=right_percent, top=0.98, bottom=0.02)

for ax, (rep1, rep2) in zip(axes, comparisons):
    print(f"generating {rep1.name}...")
    start = 400
    size = 75

        
    #     print(data.name)
    
#     p1 = rep1.worked.copy()[start:start+size]
#     p2 = rep2.worked.copy()[start:start+size]
#     e1 = rep1.worked_stderr.copy()[start:start+size]
#     e2 = rep2.worked_stderr.copy()[start:start+size]
    
#     p1 = rep1.profile.copy()[start:start+size]
#     p2 = rep2.profile.copy()[start:start+size]
#     e1 = rep1.stderr.copy()[start:start+size]
#     e2 = rep2.stderr.copy()[start:start+size]
    
    p1 = rep1.med_normed.copy()[start:start+size]
    p2 = rep2.med_normed.copy()[start:start+size]
    e1 = rep1.med_normed_stderr.copy()[start:start+size]
    e2 = rep2.med_normed_stderr.copy()[start:start+size]
    
#     e1 = [np.nan]*size
#     e2 = [np.nan]*size
#     nan_mask = np.isnan(rep1.worked)
    
#     color1 = 'royalblue'
#     color2 = 'black'

    color1 = 'green'
    color2 = 'lightgreen'
    
#     ax.bar(tuple(rep1.num[start:start+size]), p1, width=1, yerr=e1, ecolor=color1, capsize=6, color=color1, alpha=0.5)
#     ax.bar(tuple(rep2.num[start:start+size]), p2, width=1, yerr=e2, ecolor=color2, capsize=6, fc=(1, 1, 1, 0), edgecolor=color2, linewidth=2)
#      error_kw={'elinewidth':.001, 'ls':'--'}
    

    ax.step(tuple(rep1.num[start:start+size]), p1, color=color1, where='mid')
    ax.step(tuple(rep2.num[start:start+size]), p2, color=color2, where='mid')
#     ax.bar(tuple(rep1.num[start:start+size]), p1, alpha=0, yerr=e1)

    # for bar errorbars
    ax.errorbar(np.array(rep1.num[start:start+size])-.1, y=p1, yerr=e1, capsize=3, color=color1, fmt='none')
    ax.errorbar(np.array(rep2.num[start:start+size])+.1, y=p2, yerr=e2, capsize=3, color=color2, fmt='none')
    
    # for filled errorbars
#     ax.fill_between(tuple(rep1.num[start:start+size]), p1+e1, p1-e1, step='pre', color=color1, alpha=0.3)
#     ax.fill_between(tuple(rep2.num[start:start+size]), p2+e2, p2-e2, step='pre', color=color2, alpha=0.3)


    ax.set_ylim(-.2, 4)
    ax.set_title(f"{rep1.name} (blue) vs {rep2.name}")

    
    plt.savefig(f"replicate_shape_skylines_test_rev6_{start}_{end}_a549-in-combined_vs_human-ex.pdf")

# plt.scatter(hek._in_rep.med_normed, hek._in_rep.med_normed_stderr, alpha=.1, color='red')
# plt.scatter(hek._in.med_normed, hek._in.med_normed_stderr, alpha=.1, color='blue')

In [ ]:
''' secondary version of skylines code that just generates a bunch of individual skyline plot files for various regions '''
%matplotlib inline

# import seaborn as sns

comparisons = (
#     (hek._in, hek._in_rep),
#     (hek._ex, hek._ex_rep),
#     (vero._in_rep, vero._in),
#     (vero._ex_rep, vero._ex),
#     (hek._in_merged, a549._in_merged),
#     (a549._in, a549._in_rep),
#     (a549._ex, hek._ex),
#     (hek._ex_rep, a549._ex),
#     (a549._ex, hek._ex_rep),
#     (a549._in_merged, hek._ex_merged), 
#     (a549._in_merged, hek._in_merged), 
#     (a549._in_merged, human._ex), 
#     (vero._in_merged, vero._ex_merged),
    (human._ex, hek._in_merged),
)

# comparisons = ((human._ex, a549._in_merged),)
           
# regions = []

# superfold_dir = "/home/colindt/structure_stuff/Superfold"
# superfold_files = os.listdir(superfold_dir)

# datasets = {
#     'vero-in': vero._in,
#     'vero-in-replicate': vero._in_rep,
#     'vero-in-combined': vero._in_merged,
#     'vero-ex': vero._ex,
#     'vero-ex-replicate': vero._ex_rep,
#     'vero-ex-combined': vero._ex_merged,
#     'hek-in': hek._in,
#     'hek-in-replicate': hek._in_rep,
#     'hek-in-combined': hek._in_merged,
#     'hek-ex': hek._ex,
#     'hek-ex-replicate': hek._ex_rep,
#     'hek-ex-combined': hek._ex_merged,
#     'a549-in': a549._in,
#     'a549-in-replicate': a549._in_rep,
#     'a549-in-combined': a549._in_merged,
#     'a549-ex': a549._ex,
#     'a549-ex-hek-ex-rep-merged': human._ex
# }

# files = []

# for f in superfold_files:
#     if 'results' not in f:
#         continue

#     print(f"{f}...")

#     data = datasets[f.split('_')[1]]

#     # if this dataset is in our reps, we need that file
#     if data in (rep1, rep2):
#         region_files += os.path.join(f, "regions")
        
        


region_files = os.listdir("../structure_stuff/Superfold/results_a549-in-combined_median-normalized-to-a549-in.map_9329/regions")

# regions = []

# # # parse regions from region files
# for i, f in enumerate(region_files):
#     if f[-2:] == 'ct':
#         f_split = f[:-3].split('_')
#         # adding 1 to the end of the region so that I can just use its bounds to get our real region
#         regions  += [(int(f_split[4]), int(f_split[5]) + 1)]

# figure... 1?
# regions = ((2181, 2253),
#            (4015, 4249),
#            (4465, 4516),
#            (5250, 5404),
#            (5584, 5795),
#            (6054, 6465),
#            (6508, 6633))

# regions = ((800, 875),
#            (6500,6575),
#            (2400,2475),
#            (425, 500),
# #            (0, 3250)
#            (3250, 6500),
#           )

# regions = ((2181, 2253),
#            (4465, 4516),
#            (6508, 6633),)



regions = ((6250, 6325),
           (2000, 2150),)

# regions = (
#     (400, 475),
#     (3800,3875),
#     (6500,6575)
# )

# figure 2 (or is this 3?)
# regions = ((2154, 2353),
#            (4011, 4429),
#            (4431, 4521),
#            (5250, 5416),
#            (5552, 5795),
#            (6054, 6482),
#            (6508, 6633))

left_inches = 0.9
right_inches = 0.4
fig_width = 20



num_panels = len(comparisons)

fig_height = 10*num_panels

for (start, end) in regions:
    # the ideal width ratio for skyline plots is roughly 20width/75bases, so if I multiply 20/75 by the number of bases, I should get my new width
    fig_width = (20/75)*(end-start)
#     fig_width = 8.5
#     fig_height = 11
    
    fig = plt.figure(figsize=(fig_width,fig_height))
    
#     start = r[0]
#     end = r[1]
    
    #     create axes for subplots
    axes = []    
    for i in range(len(comparisons)):
        axes += [plt.subplot(num_panels, 1, i + 1)]

    left_percent = left_inches/fig_width
    right_percent = 1-right_inches/fig_width
        
    plt.subplots_adjust(hspace=0.5, left=left_percent,right=right_percent, top=0.98, bottom=0.02)

    for ax, (rep1, rep2) in zip(axes, comparisons):
        print(f"generating {start}-{end}...")
        
    #     p1 = rep1.worked.copy()[start:start+size]
    #     p2 = rep2.worked.copy()[start:start+size]
    #     e1 = rep1.worked_stderr.copy()[start:start+size]
    #     e2 = rep2.worked_stderr.copy()[start:start+size]

    #     p1 = rep1.profile.copy()[start:start+size]
    #     p2 = rep2.profile.copy()[start:start+size]
    #     e1 = rep1.stderr.copy()[start:start+size]
    #     e2 = rep2.stderr.copy()[start:start+size]

        p1 = rep1.med_normed.copy()[start:end]
        p2 = rep2.med_normed.copy()[start:end]
        e1 = rep1.med_normed_stderr.copy()[start:end]
        e2 = rep2.med_normed_stderr.copy()[start:end]
        n1 = list(rep1.num).copy()[start:end]
        n2 = list(rep2.num).copy()[start:end]
        
#         for i in range(len(p1)):
#             if np.isnan(p1[i]) or np.isnan(p2[i]):
#                 ax.axvspan(n1[i] - .4, n1[i] + .4, color='red', alpha=0.5)
        
    #     e1 = [np.nan]*size
    #     e2 = [np.nan]*size
    #     nan_mask = np.isnan(rep1.worked)

        color1 = 'darkgoldenrod'
    
#         color1 = 'black'
        color2 = 'blue'

        # vero in
#         color2 = 'darkgreen'
#         color1 = 'yellowgreen'
        
        # vero ex
#         color2 = 'red'
#         color1 = 'hotpink'
        
    #     ax.bar(tuple(rep1.num[start:start+size]), p1, width=1, yerr=e1, ecolor=color1, capsize=6, color=color1, alpha=0.5)
    #     ax.bar(tuple(rep2.num[start:start+size]), p2, width=1, yerr=e2, ecolor=color2, capsize=6, fc=(1, 1, 1, 0), edgecolor=color2, linewidth=2)
    #      error_kw={'elinewidth':.001, 'ls':'--'}


        ax.step(tuple(rep1.num[start:end]), p1, color=color1, where='mid', label=rep1.name)
        ax.step(tuple(rep2.num[start:end]), p2, color=color2, where='mid', label=rep2.name)
    #     ax.bar(tuple(rep1.num[start:start+size]), p1, alpha=0, yerr=e1)
        ax.errorbar(np.array(rep1.num[start:end])-.1, y=p1, yerr=e1, capsize=3, color=color1, fmt='none')
        ax.errorbar(np.array(rep2.num[start:end])+.1, y=p2, yerr=e2, capsize=3, color=color2, fmt='none')
    #     ax.fill_between(tuple(rep1.num[start:start+size]), p1+e1, p1-e1, step='pre', color=color1, alpha=0.3)
    #     ax.fill_between(tuple(rep2.num[start:start+size]), p2+e2, p2-e2, step='pre', color=color2, alpha=0.3)
        ax.set_ylim(-0.5, 4)
        ax.set_xlim(start-1, end+1)
        ax.set_title(f"{rep1.name} vs {rep2.name} (blue)")
    #     ax.set_facecolor('white')

        ax.legend(loc=2)
    
    if not os.path.exists("skylines"):
        os.mkdir("skylines")
    
    plt.savefig(f"plots_skylines/replicate_shape_skylines_{rep1.name}_{rep2.name}_rev6_{start}_{end}.pdf")
    plt.close('all')

print("done")
# plt.scatter(hek._in_rep.med_normed, hek._in_rep.med_normed_stderr, alpha=.1, color='red')
# plt.scatter(hek._in.med_normed, hek._in.med_normed_stderr, alpha=.1, color='blue')